In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import nltk
import unicodedata
import re
import env
import prepare
from matplotlib import style
style.use("ggplot")
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings("ignore")
from collections import Counter

In [9]:
#make df from data.json
df = pd.read_json('data.json')
df.head()

,repo,language,readme_contents
0,mcastrolab/Brazil-Covid19-e0-change,R,# Reduction in life expectancy in Brazil after...
1,jschoeley/de0anim,R,# Animated annual changes in life-expectancy\n...
2,sychi77/Thoracic_Surgery_Patient_Survival,Jupyter Notebook,# Thoracic Surgery for Lung Cancer Data Set\n ...
3,ashtad63/HackerRank-Data-Scientist-Hiring-Test,Jupyter Notebook,# HackerRank Data Scientist Hiring Test: Predi...
4,OxfordDemSci/ex2020,R,"<p align=""center"">\n <img src=""https://github..."


In [10]:
# how many rows  have readme_contents with length == 0
df[df['readme_contents'].str.len() == 0].language.value_counts()


Jupyter Notebook    10
R                    5
Stata                3
HTML                 2
Python               1
Objective-C          1
Scheme               1
Name: language, dtype: int64

In [11]:
df["no_readme_contents"] = df["readme_contents"].str.len()==0

In [12]:
# person of language with no readme_contents
df.groupby("language")["no_readme_contents"].mean().sort_values(ascending=False)

language
Objective-C         1.000000
Scheme              1.000000
Stata               0.600000
Jupyter Notebook    0.208333
R                   0.161290
HTML                0.090909
Python              0.052632
Dart                0.000000
Java                0.000000
JavaScript          0.000000
MATLAB              0.000000
Ruby                0.000000
Scala               0.000000
Shell               0.000000
Swift               0.000000
TypeScript          0.000000
Name: no_readme_contents, dtype: float64

In [13]:
# change readme_contents to original
df.rename(columns={'readme_contents':'original'}, inplace=True)
df.head()

,repo,language,original,no_readme_contents
0,mcastrolab/Brazil-Covid19-e0-change,R,# Reduction in life expectancy in Brazil after...,False
1,jschoeley/de0anim,R,# Animated annual changes in life-expectancy\n...,False
2,sychi77/Thoracic_Surgery_Patient_Survival,Jupyter Notebook,# Thoracic Surgery for Lung Cancer Data Set\n ...,False
3,ashtad63/HackerRank-Data-Scientist-Hiring-Test,Jupyter Notebook,# HackerRank Data Scientist Hiring Test: Predi...,False
4,OxfordDemSci/ex2020,R,"<p align=""center"">\n <img src=""https://github...",False


In [14]:
# df['clean'] = df['original'].apply(prepare.basic_clean)
# df['lemmas'] = df['clean'].apply(prepare.lemmatize)
# df['stemmed'] = df['clean'].apply(prepare.stem)
# df['more_clean'] = df['lemmas'].apply(prepare.remove_stopwords) #lemmas with stop words removed

In [15]:
# remove Jupyter Notebook
#df = df[df['language'] != 'Jupyter Notebook']

In [16]:
# make a list of languages with value counts above 3
# bottom_language_counts = df['language'].value_counts()<=3
# bottom_languages = bottom_language_counts[bottom_language_counts].index.tolist()
# bottom_languages
# # return df where language is in bottom_languages
# df = df[df['language'].isin(bottom_languages)]


In [17]:
def drop_low_count_languages(df, n_occurances=3):
    """
    drop languages with less than n_occurances
    """
    # get the language counts
    bottom_language_counts = df["language"].value_counts()<=n_occurances
    #make a list of languages with value counts below n_occurances
    bottom_languages = bottom_language_counts[bottom_language_counts].index.tolist()
    # drop all columns where language is in bottom_languages ('~' means not)
    df = df[~df['language'].isin(bottom_languages)]
    
    return df

 

In [18]:
df.shape

(166, 4)

In [19]:
# df = drop_low_count_languages(df)

In [20]:
# how many languages are there?
df['language'].value_counts()

Jupyter Notebook    48
R                   31
HTML                22
Python              19
JavaScript          16
Stata                5
Java                 3
Dart                 3
Scala                2
Swift                2
MATLAB               1
Shell                1
TypeScript           1
Ruby                 1
Objective-C          1
Scheme               1
Name: language, dtype: int64

In [21]:
# keep top n languages
df = prepare.keep_top_n_languages(df, n_languages=3) 


In [22]:
# how many languages are there?
df['language'].value_counts()

other               65
Jupyter Notebook    48
R                   31
HTML                22
Name: language, dtype: int64

In [23]:
# make vectorizer
tfidf = TfidfVectorizer()
# fit the vectorizer to the data and make df
X = tfidf.fit_transform(df['more_clean'])
y = df['language']

# split data into train and test
X_train, X_validate, X_test, y_train, y_validate, y_test = prepare.split_data(X, y)

KeyError: 'more_clean'

In [ ]:
train = pd.DataFrame(dict(actual=y_train))
validate = pd.DataFrame(dict(actual=y_validate))
test = pd.DataFrame(dict(actual=y_test))


In [ ]:
def baseline_accuracy(df, mode):
    """
    Calculate baseline accuracy
    """
    df['mode'] = mode
    baseline_accuracy = accuracy_score(df['actual'], df['mode'])
    return baseline_accuracy

In [ ]:
#get mode to use as baseline
mode = df.language.mode().values[0]
#get baseline_accuracy
train_baseline = baseline_accuracy(train, mode)
validate_baseline = baseline_accuracy(validate, mode)
test_baseline = baseline_accuracy(test, mode)



In [ ]:
# make a df for results
results = pd.DataFrame()
baseline_model = pd.Series({'model_number':'baseline','model_type':'baseline','solver':np.nan,'train_accuracy':train_baseline,'validate_accuracy':validate_baseline,'test_accuracy':test_baseline, 'better_than_baseline':False})
results = pd.concat([results, baseline_model],axis = 0)
results.T

,model_number,model_type,solver,train_accuracy,validate_accuracy,test_accuracy,better_than_baseline
0,baseline,baseline,NaN,0.521739,0.525,0.529412,False


In [ ]:
results.shape[1]

1

In [ ]:
# make a df for results
results = pd.DataFrame()
baseline_model = pd.Series({'model_number':'baseline','model_type':'baseline','solver':np.nan,'train_accuracy':train_baseline,'validate_accuracy':validate_baseline,'test_accuracy':test_baseline, 'better_than_baseline':False})
results = pd.concat([results, baseline_model],axis = 0)
results.T
# make more models varying solver
model_number = results.shape[1]
for solver in ['liblinear','lbfgs','newton-cg','sag','saga']:
    lm = LogisticRegression(solver=solver).fit(X_train, y_train)
    # run model on data splits
    train['predicted'] = lm.predict(X_train)
    validate['predicted'] = lm.predict(X_validate)
    test['predicted'] = lm.predict(X_test)
    # make results series to add to results df
    stats = pd.Series(
        {'model_number':model_number,
            'model_type':'LogisticRegression',
            'solver':solver,
            'train_accuracy':accuracy_score(y_train, train['predicted']),
            'validate_accuracy':accuracy_score(y_validate, validate['predicted']),
            'test_accuracy':accuracy_score(y_test, test['predicted']),
            'better_than_baseline':accuracy_score(y_validate, validate['predicted'])>validate_baseline,
            'include_jupyter_notebook':'Jupyter Notebook' in df.language
        })
    # add to results df
    results = pd.concat([results, stats], axis = 1)
    model_number += 1

results.T.reset_index(drop=True)

,model_number,model_type,solver,train_accuracy,validate_accuracy,test_accuracy,better_than_baseline,include_jupyter_notebook
0,baseline,baseline,NaN,0.521739,0.525,0.529412,False,NaN
1,1,LogisticRegression,liblinear,0.73913,0.525,0.558824,False,False
2,2,LogisticRegression,lbfgs,0.826087,0.525,0.529412,False,False
3,3,LogisticRegression,newton-cg,0.826087,0.525,0.529412,False,False
4,4,LogisticRegression,sag,0.826087,0.525,0.529412,False,False
5,5,LogisticRegression,saga,0.836957,0.525,0.529412,False,False


In [ ]:
def run_models(drop_jupyter=True, drop_low_count_langs=True, n_languages=3):
    """
    Run models on data varying solver
    """
    #get raw data
    df = pd.read_json('data.json')
    # clean data
    df['more_clean'] = df['readme_contents'].apply(prepare.more_clean)
    if drop_jupyter:
        df = df[df['language'] != 'Jupyter Notebook']
    if drop_low_count_langs:
        df = drop_low_count_languages(df)
    df = prepare.keep_top_n_languages(df, n_languages=n_languages)
    # make vectorizer
    tfidf = TfidfVectorizer()
    # fit the vectorizer to the data and make df
    X = tfidf.fit_transform(df['more_clean'])
    y = df['language']

    # split data into train and test
    X_train, X_validate, X_test, y_train, y_validate, y_test = prepare.split_data(X, y)
    train = pd.DataFrame(dict(actual=y_train))
    validate = pd.DataFrame(dict(actual=y_validate))
    test = pd.DataFrame(dict(actual=y_test))
    #get mode to use as baseline
    mode = df.language.mode().values[0]
    #get baseline_accuracy
    train_baseline = baseline_accuracy(train, mode)
    validate_baseline = baseline_accuracy(validate, mode)
    test_baseline = baseline_accuracy(test, mode)
    # make a df for results
    results = pd.DataFrame()
    # make baseline model
    baseline_model = pd.Series({'model_number':'baseline','model_type':'baseline','solver':np.nan,'train_accuracy':train_baseline,'validate_accuracy':validate_baseline,'test_accuracy':test_baseline, 'better_than_baseline':False})
    # add baseline model to results df
    results = pd.concat([results, baseline_model],axis = 0)
    # make more models varying solver
    model_number = results.shape[1]
    for solver in ['liblinear','lbfgs','newton-cg','sag','saga']:
        lm = LogisticRegression(solver=solver).fit(X_train, y_train)
        # run model on data splits
        train['predicted'] = lm.predict(X_train)
        validate['predicted'] = lm.predict(X_validate)
        test['predicted'] = lm.predict(X_test)
        # make results series to add to results df
        stats = pd.Series(
        {'model_number':model_number,
            'model_type':'LogisticRegression',
            'solver':solver,
            'train_accuracy':accuracy_score(y_train, train['predicted']),
            'validate_accuracy':accuracy_score(y_validate, validate['predicted']),
            'test_accuracy':accuracy_score(y_test, test['predicted']),
            'better_than_baseline':accuracy_score(y_validate, validate['predicted'])>validate_baseline,
            'drop_jupyter':drop_jupyter,
            'drop_low_count_languages':drop_low_count_langs,
            'n_languages':n_languages
        })
        # add to results df
        results = pd.concat([results, stats], axis = 1)
        model_number += 1

    return results.T.reset_index(drop=True)


In [ ]:
run_models(drop_jupyter=True, drop_low_count_langs=True, n_languages=3)

,model_number,model_type,solver,train_accuracy,validate_accuracy,test_accuracy,better_than_baseline,drop_jupyter,drop_low_count_languages,n_languages
0,baseline,baseline,NaN,0.303571,0.32,0.285714,False,NaN,NaN,NaN
1,1,LogisticRegression,liblinear,0.928571,0.36,0.333333,True,True,True,3
2,2,LogisticRegression,lbfgs,0.928571,0.36,0.333333,True,True,True,3
3,3,LogisticRegression,newton-cg,0.928571,0.36,0.333333,True,True,True,3
4,4,LogisticRegression,sag,0.928571,0.36,0.333333,True,True,True,3
5,5,LogisticRegression,saga,0.928571,0.36,0.333333,True,True,True,3


In [ ]:
run_models(drop_jupyter=False, drop_low_count_langs=True, n_languages=3)

,model_number,model_type,solver,train_accuracy,validate_accuracy,test_accuracy,better_than_baseline,drop_jupyter,drop_low_count_languages,n_languages
0,baseline,baseline,NaN,0.321429,0.333333,0.333333,False,NaN,NaN,NaN
1,1,LogisticRegression,liblinear,0.833333,0.444444,0.4,True,False,True,3
2,2,LogisticRegression,lbfgs,0.880952,0.416667,0.4,True,False,True,3
3,3,LogisticRegression,newton-cg,0.880952,0.416667,0.4,True,False,True,3
4,4,LogisticRegression,sag,0.880952,0.416667,0.4,True,False,True,3
5,5,LogisticRegression,saga,0.880952,0.388889,0.4,True,False,True,3


In [ ]:
run_models(drop_jupyter=False, drop_low_count_langs=False, n_languages=3)

,model_number,model_type,solver,train_accuracy,validate_accuracy,test_accuracy,better_than_baseline,drop_jupyter,drop_low_count_languages,n_languages
0,baseline,baseline,NaN,0.391304,0.4,0.382353,False,NaN,NaN,NaN
1,1,LogisticRegression,liblinear,0.73913,0.425,0.441176,True,False,False,3
2,2,LogisticRegression,lbfgs,0.815217,0.475,0.441176,True,False,False,3
3,3,LogisticRegression,newton-cg,0.815217,0.475,0.441176,True,False,False,3
4,4,LogisticRegression,sag,0.815217,0.475,0.441176,True,False,False,3
5,5,LogisticRegression,saga,0.826087,0.5,0.441176,True,False,False,3


In [ ]:
def run_models2(drop_jupyter=True, drop_low_count_langs=True, n_languages=3):
    """
    Run models on data varying solver and C value
    """
    #get raw data
    df = pd.read_json('data.json')
    # clean data
    df['more_clean'] = df['readme_contents'].apply(prepare.more_clean)
    if drop_jupyter:
        df = df[df['language'] != 'Jupyter Notebook']
    if drop_low_count_langs:
        df = drop_low_count_languages(df)
    df = prepare.keep_top_n_languages(df, n_languages=n_languages)
    # make vectorizer
    tfidf = TfidfVectorizer()
    # fit the vectorizer to the data and make df
    X = tfidf.fit_transform(df['more_clean'])
    y = df['language']

    # split data into train and test
    X_train, X_validate, X_test, y_train, y_validate, y_test = prepare.split_data(X, y)
    train = pd.DataFrame(dict(actual=y_train))
    validate = pd.DataFrame(dict(actual=y_validate))
    test = pd.DataFrame(dict(actual=y_test))
    #get mode to use as baseline
    mode = df.language.mode().values[0]
    #get baseline_accuracy
    train_baseline = baseline_accuracy(train, mode)
    validate_baseline = baseline_accuracy(validate, mode)
    test_baseline = baseline_accuracy(test, mode)
    # make a df for results
    results = pd.DataFrame()
    # make baseline model
    baseline_model = pd.Series({'model_number':'baseline','model_type':'baseline','solver':np.nan,'train_accuracy':train_baseline,'validate_accuracy':validate_baseline,'test_accuracy':test_baseline, 'better_than_baseline':False})
    # add baseline model to results df
    results = pd.concat([results, baseline_model],axis = 0)
    # make more models varying solver
    model_number = results.shape[1]
    c_values = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
    for solver in ['liblinear','lbfgs','newton-cg','sag','saga']:
        for c in c_values:
            #make the model
            lm = LogisticRegression(C=c, solver=solver).fit(X_train, y_train)
            # run model on data splits
            train['predicted'] = lm.predict(X_train)
            validate['predicted'] = lm.predict(X_validate)
            test['predicted'] = lm.predict(X_test)
            # make results series to add to results df
            stats = pd.Series(
            {'model_number':model_number,
                'model_type':'LogisticRegression',
                'solver':solver,
                'C':c,
                'train_accuracy':accuracy_score(y_train, train['predicted']),
                'validate_accuracy':accuracy_score(y_validate, validate['predicted']),
                'test_accuracy':accuracy_score(y_test, test['predicted']),
                'better_than_baseline':accuracy_score(y_validate, validate['predicted'])>validate_baseline,
                'drop_jupyter':drop_jupyter,
                'drop_low_count_languages':drop_low_count_langs,
                'n_languages':n_languages
            })
            # add to results df
            results = pd.concat([results, stats], axis = 1)
            model_number += 1

    return results.T.reset_index(drop=True)

In [ ]:
run_models2(drop_jupyter=True, drop_low_count_langs=True, n_languages=3).sort_values('validate_accuracy', ascending=False).head(3)

,model_number,model_type,solver,train_accuracy,validate_accuracy,test_accuracy,better_than_baseline,C,drop_jupyter,drop_low_count_languages,n_languages
7,7,LogisticRegression,liblinear,0.928571,0.4,0.428571,True,1000,True,True,3
18,18,LogisticRegression,newton-cg,0.928571,0.36,0.333333,True,1,True,True,3
16,16,LogisticRegression,newton-cg,0.553571,0.36,0.333333,True,0.01,True,True,3


In [ ]:
run_models2(drop_jupyter=False, drop_low_count_langs=True, n_languages=3).sort_values('validate_accuracy', ascending=False).head(3)

,model_number,model_type,solver,train_accuracy,validate_accuracy,test_accuracy,better_than_baseline,C,drop_jupyter,drop_low_count_languages,n_languages
31,31,LogisticRegression,saga,0.535714,0.555556,0.4,True,0.1,False,True,3
32,32,LogisticRegression,saga,0.880952,0.5,0.433333,True,1,False,True,3
15,15,LogisticRegression,newton-cg,0.535714,0.5,0.4,True,0.001,False,True,3


In [ ]:
run_models2(drop_jupyter=False, drop_low_count_langs=False, n_languages=3).sort_values('validate_accuracy', ascending=False).head(3)

,model_number,model_type,solver,train_accuracy,validate_accuracy,test_accuracy,better_than_baseline,C,drop_jupyter,drop_low_count_languages,n_languages
5,5,LogisticRegression,liblinear,0.913043,0.5,0.5,True,10,False,False,3
32,32,LogisticRegression,saga,0.836957,0.5,0.441176,True,1,False,False,3
18,18,LogisticRegression,newton-cg,0.815217,0.475,0.441176,True,1,False,False,3


In [ ]:
run_models2(drop_jupyter=True, drop_low_count_langs=True, n_languages=3)

,model_number,model_type,solver,train_accuracy,validate_accuracy,test_accuracy,better_than_baseline,C,drop_jupyter,drop_low_count_languages,n_languages
0,baseline,baseline,NaN,0.303571,0.32,0.285714,False,NaN,NaN,NaN,NaN
1,1,LogisticRegression,liblinear,0.553571,0.28,0.285714,False,0.001,True,True,3
2,2,LogisticRegression,liblinear,0.553571,0.28,0.285714,False,0.01,True,True,3
3,3,LogisticRegression,liblinear,0.553571,0.36,0.333333,True,0.1,True,True,3
4,4,LogisticRegression,liblinear,0.928571,0.36,0.333333,True,1,True,True,3
5,5,LogisticRegression,liblinear,0.928571,0.36,0.428571,True,10,True,True,3
6,6,LogisticRegression,liblinear,0.928571,0.36,0.428571,True,100,True,True,3
7,7,LogisticRegression,liblinear,0.928571,0.4,0.428571,True,1000,True,True,3
8,8,LogisticRegression,lbfgs,0.553571,0.36,0.333333,True,0.001,True,True,3
9,9,LogisticRegression,lbfgs,0.553571,0.36,0.333333,True,0.01,True,True,3


In [ ]:
df.head()


,repo,language,original,more_clean,char_count,word_count,unique_word_count
0,mcastrolab/Brazil-Covid19-e0-change,R,# Reduction in life expectancy in Brazil after...,reduction life expectancy brazil covid provide...,2949,352,211
1,jschoeley/de0anim,R,# Animated annual changes in life-expectancy\n...,animated annual change lifeexpectancy illustra...,166,16,16
2,sychi77/Thoracic_Surgery_Patient_Survival,Python,# Thoracic Surgery for Lung Cancer Data Set\n ...,thoracic surgery lung cancer data set uci mach...,2058,234,150
3,ashtad63/HackerRank-Data-Scientist-Hiring-Test,Python,# HackerRank Data Scientist Hiring Test: Predi...,hackerrank data scientist hiring test predict ...,1011,135,86
4,OxfordDemSci/ex2020,R,"<p align=""center"">\n <img src=""https://github...",p aligncenter img srchttpsgithubcomoxforddemsc...,2259,234,172


In [ ]:
# which word is the most common in the string
def n_most_common_word(string, n=1):
    """
    Return the most common word in a string
    """
    words = string.split()
    if len(words) < n:
        return ''
    word_counts = Counter(words)
    return word_counts.most_common(n)[n-1][0]

n_most_common_word(df.more_clean[1],3)

'change'

In [ ]:
# add column to df with most common word
df['most_common_word'] = df['more_clean'].apply(n_most_common_word)
# add column to df with 2nd common word
df['2nd_most_common_word'] = df['more_clean'].apply(n_most_common_word, n=2)
# add column to df with 3rd common word
df['3rd_most_common_word'] = df['more_clean'].apply(n_most_common_word, n=3)
# add column to df with 4th common word
df['4th_most_common_word'] = df['more_clean'].apply(n_most_common_word, n=4)
# add column to df with 5th common word
df['5th_most_common_word'] = df['more_clean'].apply(n_most_common_word, n=5)

In [ ]:
df.head()

,repo,language,original,more_clean,char_count,word_count,unique_word_count,most_common_word,2nd_most_common_word,3rd_most_common_word,4th_most_common_word,5th_most_common_word
0,mcastrolab/Brazil-Covid19-e0-change,R,# Reduction in life expectancy in Brazil after...,reduction life expectancy brazil covid provide...,2949,352,211,life,expectancy,covid,state,data
1,jschoeley/de0anim,R,# Animated annual changes in life-expectancy\n...,animated annual change lifeexpectancy illustra...,166,16,16,animated,annual,change,lifeexpectancy,illustration
2,sychi77/Thoracic_Surgery_Patient_Survival,Python,# Thoracic Surgery for Lung Cancer Data Set\n ...,thoracic surgery lung cancer data set uci mach...,2058,234,150,surgery,f,data,lung,thoracic
3,ashtad63/HackerRank-Data-Scientist-Hiring-Test,Python,# HackerRank Data Scientist Hiring Test: Predi...,hackerrank data scientist hiring test predict ...,1011,135,86,country,life,expectancy,must,test
4,OxfordDemSci/ex2020,R,"<p align=""center"">\n <img src=""https://github...",p aligncenter img srchttpsgithubcomoxforddemsc...,2259,234,172,data,relates,code,p,generic


In [ ]:
def run_models3():
    """
    Run models on data varying solver and C value
    """
    #get raw data
    df = pd.read_json('data.json')
    df = prepare.prepare_data(df)
    # make vectorizer
    tfidf = TfidfVectorizer()
    # fit the vectorizer to the data and make df
    X = tfidf.fit_transform(df['more_clean'])
    y = df['language']

    # split data into train and test
    X_train, X_validate, X_test, y_train, y_validate, y_test = prepare.split_data(X, y)
    train = pd.DataFrame(dict(actual=y_train))
    validate = pd.DataFrame(dict(actual=y_validate))
    test = pd.DataFrame(dict(actual=y_test))
    #get mode to use as baseline
    mode = df.language.mode().values[0]
    #get baseline_accuracy
    train_baseline = baseline_accuracy(train, mode)
    validate_baseline = baseline_accuracy(validate, mode)
    test_baseline = baseline_accuracy(test, mode)
    # make a df for results
    results = pd.DataFrame()
    # make baseline model
    baseline_model = pd.Series({'model_number':'baseline','model_type':'baseline','solver':np.nan,'train_accuracy':train_baseline,'validate_accuracy':validate_baseline,'test_accuracy':test_baseline, 'better_than_baseline':False})
    # add baseline model to results df
    results = pd.concat([results, baseline_model],axis = 0)
    # make more models varying solver
    model_number = results.shape[1]
    c_values = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
    for solver in ['liblinear','lbfgs','newton-cg','sag','saga']:
        for c in c_values:
            #make the model
            lm = LogisticRegression(C=c, solver=solver).fit(X_train, y_train)
            # run model on data splits
            train['predicted'] = lm.predict(X_train)
            validate['predicted'] = lm.predict(X_validate)
            test['predicted'] = lm.predict(X_test)
            # make results series to add to results df
            stats = pd.Series(
            {'model_number':model_number,
                'model_type':'LogisticRegression',
                'solver':solver,
                'C':c,
                'train_accuracy':accuracy_score(y_train, train['predicted']),
                'validate_accuracy':accuracy_score(y_validate, validate['predicted']),
                'test_accuracy':accuracy_score(y_test, test['predicted']),
                'better_than_baseline':accuracy_score(y_validate, validate['predicted'])>validate_baseline,
            })
            # add to results df
            results = pd.concat([results, stats], axis = 1)
            model_number += 1

    return results.T.reset_index(drop=True)

In [ ]:
df = pd.read_json('data.json')
df = prepare.prep_data(df)
df.head()

,repo,language,original,clean,more_clean,char_count,word_count,unique_word_count
0,mcastrolab/Brazil-Covid19-e0-change,R,# Reduction in life expectancy in Brazil after...,reduction life expectancy brazil covid provide...,reduction life expectancy brazil covid provide...,2949,352,211
1,jschoeley/de0anim,R,# Animated annual changes in life-expectancy\n...,animated annual changes lifeexpectancy illustr...,animated annual change lifeexpectancy illustra...,166,16,16
2,sychi77/Thoracic_Surgery_Patient_Survival,Python,# Thoracic Surgery for Lung Cancer Data Set\n ...,thoracic surgery lung cancer data set uci mach...,thoracic surgery lung cancer data set uci mach...,2058,234,150
3,ashtad63/HackerRank-Data-Scientist-Hiring-Test,Python,# HackerRank Data Scientist Hiring Test: Predi...,hackerrank data scientist hiring test predict ...,hackerrank data scientist hiring test predict ...,1011,135,86
4,OxfordDemSci/ex2020,R,"<p align=""center"">\n <img src=""https://github...",p aligncenter img srchttpsgithubcomoxforddemsc...,p aligncenter img srchttpsgithubcomoxforddemsc...,2259,234,172
